# load data and do elementary calculations and plots

---

In [ ]:
import os
import numpy as np
import dask
from dask_jobqueue import PBSCluster
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

#import xrft

#from utils import *
from mitequinox.utils import *

# debug
from datetime import datetime
from time import sleep

In [ ]:
# launch cluster and workers
#local_dir = os.getenv('HOME')+'/dask_scratch/'
local_dir = os.getenv('DATAWORK')+'/dask_scratch/'
#cluster = PBSCluster(queue='mpi_1', local_directory=os.getenv('DATAWORK')+'/dask', interface='ib0', \
cluster = PBSCluster(queue='mpi_1', local_directory=local_dir, interface='ib0', walltime='24:00:00',
                     threads=14, processes=2, memory='50GB', resource_spec='select=1:ncpus=28:mem=100g', 
                     death_timeout=100)

# original:
#                     threads=4, processes=7, memory='10GB', resource_spec='select=1:ncpus=14:mem=100g', 
#
#                     job_extra=['Output_Path='+local_dir+'output.o', 'Error_Path='+local_dir+'output.e'])
#job_extra=['Output_Path='+local_dir+'output.o', 'Error_Path='+local_dir+'output.e'])

# try with memory=5GB instead of 10GB - workers die
# try with threads=2 instead of 4 - workers die
# 


In [ ]:
print(cluster.job_header)

In [ ]:
cluster.start_workers(40)

In [ ]:
while True:
    print(datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
    print(cluster.scheduler)
    sleep(2)

In [ ]:
cluster.scheduler

In [ ]:
### get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)
#client = Client(scheduler_file=os.path.expanduser('/home1/scratch/aponte/dask/scheduler.json'), timeout='20s')
# upload utils.py to workers
#client.upload_file('utils.py')

In [ ]:
client

In [ ]:

data_dir = '/home/datawork-lops-osi/data/mit4320/Eta/'
grid_dir = '/home/datawork-lops-osi/data/mit4320/grid/'

iters, time = get_iters_time('Eta', data_dir, delta_t=25.)

ds = get_compressed_data('Eta', data_dir, grid_dir, iters='all', time=time, client=client)
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))


In [ ]:
ds['Eta'].isel(face=1, time=0).plot()

In [ ]:
ds['Eta'].isel(face=1, time=0, i=slice(0,100), j=slice(0,100)).plot()

In [ ]:
ds['Eta'].isel(face=1, i=0, j=0).mean().compute()

In [ ]:
ds['Eta'].isel(face=1, i=0, j=0, time=slice(0,2000)).plot()
#.chunk({'time': 4096})

In [ ]:
#%time ds['Eta'].isel(face=1, i=0, j=0).plot()
#%time ds['Eta'].isel(face=1, i=0, j=0).chunk({'time': 4096}).plot()

In [ ]:
%time ds['Eta'].isel(face=1, i=slice(0,100), j=slice(0,100)).mean(dim=['i','j']).plot()

---

# basic statistics

In [ ]:
m1 = ds['Eta'].isel(face=1).mean(['time']).persist()

In [ ]:
m1.plot()

---

In [ ]:
client

In [ ]:
client.restart()

In [ ]:
client.close()
cluster.stop_workers(cluster.jobs)